# Multi-View-Majority-Vote-Learning-Algorithms-Direct-Minimization-of-PAC-Bayesian-Bounds

This Notebook contains everything necessary to reproduce the experiments in our paper:  

*Multi-View Majority Vote Learning Algorithms: Direct Minimization of PAC-Bayesian Bounds*

## Imports

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from collections import OrderedDict

# Scikit-learn
from sklearn import preprocessing
from sklearn.utils import check_random_state
RAND = check_random_state(42)

# torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch.autograd import Variable
from torch.utils.data import DataLoader,Dataset
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


from mvpb.dNDF import MultiViewBoundsDeepNeuralDecisionForests


# Import data
from mvlearn.datasets import load_UCImultifeature
from data.datasets import (SampleData,
                           MultipleFeatures,
                           Nutrimouse,
                           train_test_split,
                           train_test_merge,
                           s1_s2_split)
from mvpb.util import uniform_distribution

## Load and prepare the multiview datasets

In [2]:
dataset = MultipleFeatures()
X_train, y_train, X_test, y_test = dataset.get_data()
np.unique(y_train)
np.unique(y_test)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [3]:
Xs_train = []
Xs_test = []
for xtr, xts in zip(X_train, X_test):
    scaler = preprocessing.MinMaxScaler().fit(xtr)
    Xs_train.append(scaler.transform(xtr))
    Xs_test.append(scaler.transform(xts))

X_train_concat = np.concatenate(Xs_train, axis=1)
X_test_concat = np.concatenate(Xs_test, axis=1)

In [4]:
mvb = MultiViewBoundsDeepNeuralDecisionForests(nb_estimators=3,
                                               nb_views=len(Xs_train),
                                               depth =3,
                                               used_feature_rate=0.8,
                                               epochs=5)

In [5]:
mvb.fit(Xs_train,y_train)

View 1/6 done!
View 2/6 done!
View 3/6 done!
View 4/6 done!
View 5/6 done!
View 6/6 done!


MultiViewBoundsDeepNeuralDecisionForests(depth=3, epochs=5, nb_estimators=3,
                                         nb_views=6,
                                         posterior_rho=tensor([0.3333, 0.3333, 0.3333]),
                                         used_feature_rate=0.8)

In [6]:
posterior_Qv , posterior_rho = mvb.optimize_rho('TND', unlabeled_data=Xs_test)

Iteration: 0,	 Loss: 3.3923104596291167
Iteration: 1,	 Loss: 3.391657193561021
Iteration: 2,	 Loss: 3.39041646925649
Iteration: 3,	 Loss: 3.388645773306904
Iteration: 4,	 Loss: 3.386398041915726
Iteration: 5,	 Loss: 3.3837200461851236
Iteration: 6,	 Loss: 3.380654908970559
Iteration: 7,	 Loss: 3.377238777545021
Iteration: 8,	 Loss: 3.373507791360955
Iteration: 9,	 Loss: 3.369490981561252
Iteration: 10,	 Loss: 3.3652168428074614
Iteration: 11,	 Loss: 3.3607100218594783
Iteration: 12,	 Loss: 3.3559928190870223
Iteration: 13,	 Loss: 3.35108563502967
Iteration: 14,	 Loss: 3.3460057299759423
Iteration: 15,	 Loss: 3.340770182895422
Iteration: 16,	 Loss: 3.3353937277814687
Iteration: 17,	 Loss: 3.329889335605235
Iteration: 18,	 Loss: 3.324268689216435
Iteration: 19,	 Loss: 3.3185430365000803
Iteration: 20,	 Loss: 3.312721788518111
Iteration: 21,	 Loss: 3.306814326217494
Iteration: 22,	 Loss: 3.3008272997610923
Iteration: 23,	 Loss: 3.2947682356756633
Iteration: 24,	 Loss: 3.2886438879986013
I

In [7]:
mvb.mv_tandem_risk()

(array([0.77821328, 0.45158646, 0.77792066, 0.39585682, 0.66807003,
        0.83230567]),
 0.24332873788445253,
 178.0)

In [8]:
_, mv_risk = mvb.predict_MV(Xs_test,y_test)
mv_risk

0.45999999999999996

In [9]:
lamb_bound, lamb_views = mvb.bound(bound="TND", unlabeled_data=Xs_test)

self.posterior_rho=tensor([0.0390, 0.1793, 0.0784, 0.5870, 0.0500, 0.0663]),  prior_pi=tensor([0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667])
KL_rhopi=tensor(0.4999),  KL_QP=tensor(0.0029)


In [11]:
lamb_bound, lamb_views

(1.0, [1.0, 1.0, 1.0, 1.0, 1.0, 1.0])

In [12]:
mvb.bound(bound="PBkl")

self.posterior_rho=tensor([0.0390, 0.1793, 0.0784, 0.5870, 0.0500, 0.0663]),  prior_pi=tensor([0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667])
KL_rhopi=tensor(0.4999),  KL_QP=tensor(0.0029)


(1.3602543965089613,
 [1.841217498272926,
  1.3644833404701457,
  1.7845165888890517,
  1.1613902839631476,
  1.7150574128507574,
  1.8502197858934608])

In [11]:
print(posterior_Qv[0])
print(posterior_Qv[1])
print(posterior_Qv[2])
print(posterior_Qv[3])
print(posterior_Qv[4])
print(posterior_Qv[5])

tensor([0.3539, 0.3318, 0.3143])
tensor([0.3519, 0.2919, 0.3562])
tensor([0.3207, 0.3116, 0.3678])
tensor([0.3326, 0.3327, 0.3347])
tensor([0.3377, 0.3328, 0.3295])
tensor([0.3230, 0.3430, 0.3340])


In [12]:
posterior_rho

tensor([0.1537, 0.1496, 0.1604, 0.2218, 0.1695, 0.1449])